# Maximal Bipartite Matching for Inexact Matches Between 2 Tables

#### Goal: Explore some interesting problems around using PC's to model fuzzy joins 

#### Some examples: inexact matches between tables, finding the highest and lowest possible aggregate after a match)

### Task: Build a maximal bipartite matching algorithm

### Step 1: Define a bipartite graph. (Assume we are given any similarity metric - this similarity metric is taken from geeksforgeeks).

In [4]:
def editDistance(str1, str2, m, n): 
  
    # If first string is empty, the only option is to 
    # insert all characters of second string into first 
    if m == 0: 
         return n 
  
    # If second string is empty, the only option is to 
    # remove all characters of first string 
    if n == 0: 
        return m 
  
    # If last characters of two strings are same, nothing 
    # much to do. Ignore last characters and get count for 
    # remaining strings. 
    if str1[m-1]== str2[n-1]: 
        return editDistance(str1, str2, m-1, n-1) 
  
    # If last characters are not same, consider all three 
    # operations on last character of first string, recursively 
    # compute minimum cost for all three operations and take 
    # minimum of three values. 
    return 1 + min(editDistance(str1, str2, m, n-1),    # Insert 
                   editDistance(str1, str2, m-1, n),    # Remove 
                   editDistance(str1, str2, m-1, n-1)    # Replace 
                   ) 

In [5]:
import networkx as nx

def similarity(x,y):
    return editDistance(x,y,len(x),len(y))

def construct_graph(table_a, table_b):
    bipartite_graph = nx.Graph()
    for first in table_a:
        key1, val1 = first
        id1 = key1 + '_' + str(val1) + '_1'
        for second in table_b:
            key2, val2 = second
            id2 = key2 + '_' + str(val2) + '_2' #add value to identifier to disitnguish two entries with different values
            bipartite_graph.add_edge(id1, id2, weight=1/(1+similarity(key1,key2))) #edit distance and weight should be inv. prop.
                                                                                    #also adding 1 to denom. to prevent divide by 0
            # add 1,2 to distinguish two key-value tuples belonging to different tables
    return bipartite_graph

# import dictionary for graph 
from collections import defaultdict 

# defaultdict allows that if a key is not found in the dictionary, 
# then instead of a KeyError being thrown, a new entry is created.
table_a = [('US', '300 M'), ('CN', '300 M'), ('CA', '300 M'), ('AU', '300 M'),('USA', '35 T')] 
table_b = [('USA', '35 T'), ('USA', '32 T'), ('UK', '3 T'), ('AUS', '20 T'), ('CAL', '22 T')]

### Tests:

#### Table A:

US -> 300 M

CN -> 12 B

CA -> 50 M

AU -> 25 M

#### Table B

USA -> 35 T

UK -> 3 T

USA -> 32 T

AUS -> 20 T

CAL -> 22 T

table_a = { "US" : ["300 M"],
          "CN" : ["12 B"],
          "CA" : ["50 M"],
          "AU" : ["25 M"]
        } 
        
table_b = { "USA" : ["35 T"],
          "UK" : ["3 T"],
          "USA" : ["32 T"],
          "AUS" : ["20 T"],
          "CAL" : ["22 T"]
        } 
        
matching = { "US" : "USA",
          "CN" : "",
          "CA" : "CAL",
          "AU" : "AUS",
        }
       

In [6]:
construct_graph(table_a,table_b).edges.data()
bipartite_graph = construct_graph(table_a,table_b)

## Step 2: Detail possible edge cases for maximal bipartite matching

In [7]:
# 1. both tables have the same key-val pair like USA - 35 T
# 2. table_a=(US,UK), table_b=(USA,US). natural matching would be US-USA, UK-US 
# but similarity metric makes the other matching just as likely?

## Step 3: Maximal Bipartite Matching Algorithm

In [8]:
nx.algorithms.matching.max_weight_matching(bipartite_graph)

{('AUS_20 T_2', 'AU_300 M_1'),
 ('CA_300 M_1', 'CAL_22 T_2'),
 ('CN_300 M_1', 'UK_3 T_2'),
 ('USA_35 T_1', 'USA_32 T_2'),
 ('US_300 M_1', 'USA_35 T_2')}

# Updates after meeting 03.25.2020

## Tasks

Generalize the code to 

1. allow for an arbitrary weight function and handle both maximum and minimum

2. allow to be given two dataframes, and a function that gives you correspondence.

In [61]:
import pandas as pd


"""

Transforms the given file to a pandas dataframe object if it was not one already
Assumption: Assumes that the data starts from the 1st row of given file, does not use seperators such as "," or ";"

Input: Any file
Output: A pandas dataframe object
"""
def convert_df(file):
    if isinstance(file, pd.DataFrame):
        return file
    else:
        df = pd.read_csv(file)
        return df
"""

Calculates maximum weight for the matching

Input: keys from 2 tables
Output: weight for each matching to be used in the weight part of constructing the graph
"""
def calc_max_weight(key1, key2):
    weight = 1/(1+similarity(key1,key2))
    return weight

"""

Calculates minimum weight for the matching

Input: keys from 2 tables
Output: weight for each matching to be used in the weight part of constructing the graph
"""
def calc_min_weight(key1, key2):
    weight = (-1)/(1+similarity(key1,key2))
    return weight

"""

Converts the dataframe into dictionary for better accuracy matching of pairs. 
Assumption: The data has headers in the first row (description of what that column describes)

Input: Any file
Output: A dictionary in the form col1:col2 matching
"""
def make_dict(file):
    V = list(file.to_dict('list').values())
    keys = V[0]
    values = zip(*V[1:])
    table = dict(zip(keys,values))
    return table
            
"""

Constructs a maximal bipartite graph of the given two tables

Input: Any 2 files in any format
Output: A Bipartite Graph with Maximal Weights
"""
def updated_maximal_construct_graph(file_a, file_b):
    # The area marked with ------ can be made into a helper function to avoid repetition
    table_a_unprocessed = convert_df(file_a)
    table_b_unprocessed = convert_df(file_b)
    bipartite_graph = nx.Graph()
    
    table_a = make_dict(table_a_unprocessed)
    table_b = make_dict(table_b_unprocessed)
    print(table_a)
    print(table_b)
    for key1, val1 in table_a.items():
       # print(val1)
       # key1, val1 = first
        id1 = key1 + '_' + str(val1) + '_1'
        for key2, val2 in table_b.items():
          #  key2, val2 = second
            id2 = key2 + '_' + str(val2) + '_2' #add value to identifier to disitnguish two entries with different values
            bipartite_graph.add_edge(id1, id2, weight=calc_max_weight(key1, key2)) #edit distance and weight should be inv. prop.
                                                                                    #also adding 1 to denom. to prevent divide by 0
            # add 1,2 to distinguish two key-value tuples belonging to different tables
    return bipartite_graph

"""

Constructs a maximal bipartite graph of the given two tables

Input: Any 2 files in any format
Output: A Bipartite Graph with Minimal Weights
"""
def updated_minimal_construct_graph(file_a, file_b):
    table_a_unprocessed = convert_df(file_a)
    table_b_unprocessed = convert_df(file_a)
    bipartite_graph = nx.Graph()
    
    table_a = make_dict(table_a_unprocessed)
    table_b = make_dict(table_b_unprocessed)
    for key1, val1 in table_a.items():
     #   key1, val1 = first
        id1 = key1 + '_' + str(val1) + '_1'
        for key2, val2 in table_b.items():
         #   key2, val2 = second
            id2 = key2 + '_' + str(val2) + '_2' #add value to identifier to disitnguish two entries with different values
            bipartite_graph.add_edge(id1, id2, weight=calc_min_weight(key1, key2)) #edit distance and weight should be inv. prop.
                                                                            #also adding 1 to denom. to prevent divide by 0
                                     # add 1,2 to distinguish two key-value tuples belonging to different tables
    return bipartite_graph

bipartite_graph_maximal = updated_maximal_construct_graph("table_a.csv","table_b.csv")
bipartite_graph_maximal
bipartite_graph_minimal = updated_minimal_construct_graph("table_a.csv", "table_b.csv")
bipartite_graph_minimal

{'US': ('300 M', 1), 'CN': ('12 B', 2), 'CA': ('50 M', 3), 'AU': ('25 M', 4)}
{'USA': ('32 T', 7), 'UK': ('3 T', 5), 'AUS': ('20 T', 8), 'CAL': ('22 T', 9)}


In [62]:
nx.algorithms.matching.max_weight_matching(bipartite_graph_maximal)

{("AU_('25 M', 4)_1", "AUS_('20 T', 8)_2"),
 ("CAL_('22 T', 9)_2", "CA_('50 M', 3)_1"),
 ("CN_('12 B', 2)_1", "UK_('3 T', 5)_2"),
 ("USA_('32 T', 7)_2", "US_('300 M', 1)_1")}

In [63]:
nx.algorithms.matching.max_weight_matching(bipartite_graph_minimal)

set()